# 使用套件

In [12]:
import datetime as dt
import requests
from bs4 import BeautifulSoup
from datetime import timedelta, date
import numpy as np
import pandas as pd
import pickle

# 定義一個爬選擇權資料的函數

In [13]:
def GetTodayOption(year, month, day):
    keys = {'syear': year, 'smonth': month, 'sday': day} # 要傳給期貨交易所的 key
    r = requests.get("http://www.taifex.com.tw/chinese/3/3_2_1.asp", data = keys) # 抓取這個 key 回傳網頁的原始碼
    r.encoding='utf8'
    soup = BeautifulSoup(r.text , "html.parser") # 把原始碼做整理
    soup_data1 = pd.read_html(str(soup.select('table')[2]),header=0)[2] ## 我不知道為什麼最後是[2]
    soup_data2 = pd.read_html(str(soup.select('table')[5]),header=0)[2]
    soup_data1=soup_data1.drop(['最高','最低','結算價','成交量','未平倉','漲跌'],axis=1)
    soup_data2=soup_data2.drop(['到期月份(週期)','最高','最低','結算價','成交量','未平倉','漲跌','履約價'],axis=1)
    df = pd.concat([soup_data1,soup_data2],axis=1).replace('-','NaN').convert_objects(convert_numeric=True)
    df.columns = ['到期月份','履約價','Call_最後成交價','Put_最後成交價']
    
    
    df['需刪除履約價'] = pd.DataFrame(df['履約價']%100)
    df['compare'] =df['Call_最後成交價'] - df['Put_最後成交價']
    
    df = df[df.需刪除履約價 != 50]
    
    df = pd.concat([df['到期月份'],df['履約價'],df['Call_最後成交價'],df['Put_最後成交價'],df['compare']],axis=1)
    df = df.reset_index(drop=True)
    
    return df

In [14]:
GetTodayOption(2017,1,6)

C:\Users\ericlin\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  # Remove the CWD from sys.path while we load stuff.


,到期月份,履約價,Call_最後成交價,Put_最後成交價,compare
0,NaN,9200,179.0,2.7,176.3
1,NaN,9300,86.0,11.5,74.5
2,NaN,9400,23.0,48.0,-25.0
3,NaN,9500,2.0,131.0,-129.0
4,NaN,9600,0.3,225.0,-224.7
5,201701.0,8400,950.0,0.4,949.6
6,201701.0,8500,NaN,0.5,NaN
7,201701.0,8600,NaN,0.8,NaN
8,201701.0,8700,670.0,1.1,668.9
9,201701.0,8800,565.0,1.5,563.5


# 定義一個輸出每個月第一個星期三的日期的函數

In [15]:
def  first_wen(year,month):                         # 此函數需參數 年 及 月
    day=7-(dt.date(year,month,1).weekday()+4)%7    # weekday函數 禮拜一為0;禮拜日為6
    return year,month,day

d = {}
for i in range(2010,2018):
    for j in range(1,13):
        d[i,j] = first_wen(i,j)
res = pickle.dumps(d)

B = pd.DataFrame.from_dict(d,orient='index')
B.columns = ['年','月','日']
date = pd.concat([B['年'],B['月'],B['日']],axis=1)
year = list(B['年'])
month = list(B['月'])
day =list(B['日'])

# 定義一個輸出每個月第三個星期三的日期的函數

In [16]:
def  third_wen(year_2,month_2):                         # 此函數需參數 年 及 月
    day_2=21-(dt.date(year_2,month_2,1).weekday()+4)%7    # weekday函數 禮拜一為0;禮拜日為6
    return year_2,month_2,day_2

d = {}
for i in range(2010,2018):
    for j in range(1,13):
        d[i,j] = third_wen(i,j)
res = pickle.dumps(d)

A = pd.DataFrame.from_dict(d,orient='index')#.convert_objects(convert_numeric=True)
A.columns = ['年','月','日']
date = pd.concat([A['年'],A['月'],A['日']],axis=1)
year_2 = list(A['年'])
month_2 = list(A['月'])
day_2 =list(A['日'])

# 寫一個迴圈，並且輸出成dataframe

In [17]:
df = pd.DataFrame()
df_2= pd.DataFrame()

for i in range(len(year)):
    
    buy = GetTodayOption(year[0+i],month[0+i],day[0+i])
    
    timeup = GetTodayOption(year_2[0+i],month_2[0+i],day_2[0+i])
    
    
    print(i)
    
    try:
        
        date=np.isnan(buy['到期月份'])
        date=list(date)
        loc=date.index(0)
        loc
        date=buy['到期月份'][loc]
        index=buy['到期月份']==date
        buy=buy[index]
        buy=buy.reset_index(drop=True)

        date_2=np.isnan(timeup['到期月份'])
        date_2=list(date_2)
        loc=date_2.index(0)
        loc
        date_2=timeup['到期月份'][loc]
        index=timeup['到期月份']==date_2
        timeup=timeup[index]    
        timeup=timeup.reset_index(drop=True)

        #抓出買入時的價平            
        Cmp = buy['compare']
        Cmp = abs(Cmp)
        Cmp.min(skipna=True)
        loc = abs(Cmp)==Cmp.min(skipna=True)
        loc=list(loc)
        loc=loc.index(1)
        
        
        
        
        #timeup = timeup.filter(like=value,axis=0)
        
        
        ######################################################################################################################
        Cmp = buy[(loc-4):(loc+5)]   ####可以改要買價外價評價內幾檔 ##########
        ######################################################################################################################
        
        Cmp_2 = timeup['compare']
        Cmp_2 = abs(Cmp_2)
        Cmp_2.min(skipna=True)
        loc = abs(Cmp_2)==Cmp_2.min(skipna=True)
        loc=list(loc)
        loc=loc.index(1)
        
        
        Cmp_2 = timeup[(loc-4):(loc+5)]

        
        
        df=pd.concat([df,Cmp])
        df_2=pd.concat([df_2,Cmp_2])
        
    except Exception as e:
        print ('stock market was not open on %d/%d/%d' %(year[0+i], month[0+i], day[0+i]))
        pass
    

C:\Users\ericlin\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  # Remove the CWD from sys.path while we load stuff.


0
stock market was not open on 2010/1/6
1
stock market was not open on 2010/2/3
2
stock market was not open on 2010/3/3
3
stock market was not open on 2010/4/7
4
stock market was not open on 2010/5/5
5
stock market was not open on 2010/6/2
6
stock market was not open on 2010/7/7
7
stock market was not open on 2010/8/4
8
stock market was not open on 2010/9/1
9
stock market was not open on 2010/10/6
10
stock market was not open on 2010/11/3
11
stock market was not open on 2010/12/1
12
stock market was not open on 2011/1/5
13
stock market was not open on 2011/2/2
14
stock market was not open on 2011/3/2
15
stock market was not open on 2011/4/6
16
stock market was not open on 2011/5/4
17
stock market was not open on 2011/6/1
18
stock market was not open on 2011/7/6
19
stock market was not open on 2011/8/3
20
stock market was not open on 2011/9/7
21
stock market was not open on 2011/10/5
22
stock market was not open on 2011/11/2
23
stock market was not open on 2011/12/7
24
stock market was 

In [ ]:
df

In [ ]:
df_2

# 刪除df與df_2不重疊的資料

df=df.replace(201002.0,'NaN').convert_objects(convert_numeric=True).dropna()
df=df.replace(201006.0,'NaN').convert_objects(convert_numeric=True).dropna()
df=df.replace(201308.0,'NaN').convert_objects(convert_numeric=True).dropna()
df=df.replace(201502.0,'NaN').convert_objects(convert_numeric=True).dropna()
df

df_2=df_2.replace(201102.0,'NaN').convert_objects(convert_numeric=True).dropna()
df_2=df_2.replace(201204.0,'NaN').convert_objects(convert_numeric=True).dropna()
df_2=df_2.replace(201305.0,'NaN').convert_objects(convert_numeric=True).dropna()
df_2=df_2.replace(201401.0,'NaN').convert_objects(convert_numeric=True).dropna()
df_2=df_2.replace(201702.0,'NaN').convert_objects(convert_numeric=True).dropna()
df_2=df_2.replace(201710.0,'NaN').convert_objects(convert_numeric=True).dropna()
df_2

# 存成CSV檔案

In [9]:
df.to_csv('buy.csv')

In [10]:
df_2.to_csv('ontime.csv')